# Chonkie + Qdrant Integration - Complete Guide

This notebook demonstrates the **QdrantHandshake** in Chonkie - seamless integration with Qdrant vector database for RAG applications.

## What is QdrantHandshake?

QdrantHandshake provides a simple interface for:
- ✅ Storing chunked documents in Qdrant
- ✅ Automatic embedding generation
- ✅ Semantic search with natural language queries
- ✅ Integration with Chonkie's Pipeline API
- ✅ Support for both local and cloud Qdrant instances

## Key Features:
- 🚀 **In-memory Qdrant server** - No Docker required
- 🔍 **Semantic search** - Natural language queries
- 🔗 **Pipeline integration** - Fluent API with `.store_in()`
- 🎯 **Custom embeddings** - Use any sentence-transformers model
- ☁️ **Cloud support** - Works with Qdrant Cloud
- 📦 **Automatic collection management** - Creates collections automatically

## Visual Overview

```mermaid
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#ff6b6b','primaryTextColor':'#fff','primaryBorderColor':'#c92a2a','lineColor':'#339af0','secondaryColor':'#51cf66','tertiaryColor':'#ffd43b','background':'#f8f9fa','mainBkg':'#e3fafc','secondBkg':'#fff3bf','tertiaryBkg':'#ffe3e3','textColor':'#212529','fontSize':'16px'}}}%%

graph TB
    Start([🚀 Chonkie + Qdrant<br/>Integration]):::startClass
    
    Start --> Chunker["📄 Chunker<br/>Create Chunks"]:::chunkerClass
    
    Chunker --> Embeddings["🧮 Embedding Model<br/>Generate Vectors"]:::embedClass
    
    Embeddings --> Handshake["🤝 QdrantHandshake<br/>Store & Search"]:::handshakeClass
    
    Handshake --> Storage{Storage Options}:::decisionClass
    
    Storage -->|Local| InMemory["💾 In-Memory<br/>Fast Development"]:::localClass
    Storage -->|Local| Disk["💿 Persistent<br/>Local Storage"]:::diskClass
    Storage -->|Cloud| QdrantCloud["☁️ Qdrant Cloud<br/>Production"]:::cloudClass
    
    InMemory --> Operations[Operations]:::opsClass
    Disk --> Operations
    QdrantCloud --> Operations
    
    Operations --> Write["✍️ Write<br/>Store Chunks"]:::writeClass
    Operations --> Search["🔍 Search<br/>Query Vectors"]:::searchClass
    Operations --> Delete["🗑️ Delete<br/>Remove Data"]:::deleteClass
    
    Write --> Complete([✨ RAG Ready]):::finalClass
    Search --> Complete
    Delete --> Complete
    
    classDef startClass fill:#4c6ef5,stroke:#364fc7,stroke-width:3px,color:#fff
    classDef chunkerClass fill:#fa5252,stroke:#e03131,stroke-width:2px,color:#fff
    classDef embedClass fill:#20c997,stroke:#087f5b,stroke-width:2px,color:#fff
    classDef handshakeClass fill:#ff6b6b,stroke:#c92a2a,stroke-width:3px,color:#fff
    classDef decisionClass fill:#ffd43b,stroke:#fab005,stroke-width:2px,color:#333
    classDef localClass fill:#7950f2,stroke:#5f3dc4,stroke-width:2px,color:#fff
    classDef diskClass fill:#845ef7,stroke:#6741d9,stroke-width:2px,color:#fff
    classDef cloudClass fill:#339af0,stroke:#1971c2,stroke-width:2px,color:#fff
    classDef opsClass fill:#51cf66,stroke:#37b24d,stroke-width:2px,color:#fff
    classDef writeClass fill:#ff922b,stroke:#e8590c,stroke-width:2px,color:#fff
    classDef searchClass fill:#74c0fc,stroke:#1c7ed6,stroke-width:2px,color:#fff
    classDef deleteClass fill:#ffa8a8,stroke:#fa5252,stroke-width:2px,color:#fff
    classDef finalClass fill:#4c6ef5,stroke:#364fc7,stroke-width:3px,color:#fff
```

## Installation

Install Chonkie with Qdrant support:

In [1]:
# Install chonkie with qdrant support
# !pip install "chonkie[qdrant]"

# Verify installation
try:
    from chonkie import QdrantHandshake, SemanticChunker, Pipeline
    from qdrant_client import QdrantClient
    print("✅ Chonkie with Qdrant support installed successfully!")
    print(f"  QdrantHandshake: {QdrantHandshake}")
    print(f"  QdrantClient: {QdrantClient}")
except ImportError as e:
    print(f"❌ Installation required: pip install 'chonkie[qdrant]'")
    print(f"   Error: {e}")

✅ Chonkie with Qdrant support installed successfully!
  QdrantHandshake: <class 'chonkie.handshakes.qdrant.QdrantHandshake'>
  QdrantClient: <class 'qdrant_client.qdrant_client.QdrantClient'>


## Setup In-Memory Qdrant Server

Start a local in-memory Qdrant instance - no Docker required!

In [2]:
from qdrant_client import QdrantClient

# Initialize in-memory Qdrant server
# This runs entirely in Python - perfect for development and testing
qdrant_client = QdrantClient(":memory:")

print("✅ In-Memory Qdrant Server Started!")
print(f"  Client: {qdrant_client}")
print(f"  Location: In-Memory (ephemeral)")
print(f"  Perfect for: Development, testing, notebooks")
print("\n💡 Note: Data will be lost when the notebook kernel restarts")
print("   For persistent storage, use: QdrantClient(path='./qdrant_storage')")

✅ In-Memory Qdrant Server Started!
  Client: <qdrant_client.qdrant_client.QdrantClient object at 0x00000233F443FB60>
  Location: In-Memory (ephemeral)
  Perfect for: Development, testing, notebooks

💡 Note: Data will be lost when the notebook kernel restarts
   For persistent storage, use: QdrantClient(path='./qdrant_storage')


---

# Part 1: Basic QdrantHandshake Usage

## 1. Simple Write and Search

Create chunks, store them in Qdrant, and perform semantic search.

In [5]:
import ipywidgets as widgets
from IPython.display import display

## Embedding Models for Semantic Chunking

# Popular embedding models optimized for semantic similarity tasks
semantic_embedding_models = [
    "minishlab/potion-base-32M",           # Model2Vec - Ultra-fast, 32M params
    "sentence-transformers/all-MiniLM-L6-v2",  # Popular, balanced performance
    "sentence-transformers/all-mpnet-base-v2",  # High quality, general purpose
    "BAAI/bge-small-en-v1.5",               # BGE - Efficient, strong performance
    "BAAI/bge-base-en-v1.5",                # BGE - Better quality, slower
    "BAAI/bge-large-en-v1.5",               # BGE Large - Highest quality BGE
    "thenlper/gte-small",                   # GTE - General text embeddings
    "jinaai/jina-embeddings-v2-base-en",    # Jina v2 - 8K context, strong performance
    "nomic-ai/nomic-embed-text-v1.5",       # Nomic - Long context (8K tokens)
    "emilyalsentzer/Bio_ClinicalBERT",      # MedEmbed - Medical/clinical domain
    "kamalkraj/biobert-base-cased-v1.2",    # ClinVec - Biomedical text embeddings
    "google/gecko-text-embedding",          # Google Gecko - Multimodal embeddings
    "Alibaba-NLP/gte-large-en-v1.5",        # GTE Large - State-of-the-art, 1024 dims
    "Alibaba-NLP/gte-Qwen2-7B-instruct",    # GTE Qwen2 - Instruction-tuned, 7B params
]

print("🧠 Embedding Models for SemanticChunker:")
print("=" * 70)
for idx, model in enumerate(semantic_embedding_models, 1):
    model_descriptions = {
        "minishlab/potion-base-32M": "Model2Vec - Ultra-fast, lightweight (32M params)",
        "sentence-transformers/all-MiniLM-L6-v2": "MiniLM - Popular, balanced speed/quality",
        "sentence-transformers/all-mpnet-base-v2": "MPNet - High quality semantic search",
        "BAAI/bge-small-en-v1.5": "BGE Small - Efficient, strong performance",
        "BAAI/bge-base-en-v1.5": "BGE Base - Better quality, more compute",
        "BAAI/bge-large-en-v1.5": "BGE Large - Top-tier quality, 1024 dimensions",
        "thenlper/gte-small": "GTE - General text embeddings",
        "jinaai/jina-embeddings-v2-base-en": "Jina v2 - 8K context, bilingual support",
        "nomic-ai/nomic-embed-text-v1.5": "Nomic - Long context support (8K tokens)",
        "emilyalsentzer/Bio_ClinicalBERT": "MedEmbed - Specialized for medical/clinical text",
        "kamalkraj/biobert-base-cased-v1.2": "ClinVec - BioBERT for biomedical literature",
        "google/gecko-text-embedding": "Google Gecko - Multimodal, high quality",
        "Alibaba-NLP/gte-large-en-v1.5": "GTE Large - State-of-the-art, 1024 dimensions",
        "Alibaba-NLP/gte-Qwen2-7B-instruct": "GTE Qwen2 - Instruction-tuned, 7B params",
    }
    print(f"{idx}. {model}")
    print(f"   → {model_descriptions[model]}")

print("   • Best quality: all-mpnet-base-v2, BAAI/bge-large-en-v1.5, Alibaba-NLP/gte-large-en-v1.5")
print("   • Faster models: minishlab/potion-base-32M, thenlper/gte-small")
print("   • Large-scale: Alibaba-NLP/gte-Qwen2-7B-instruct (7B params, instruction-tuned)")
print("   • Default recommended: minishlab/potion-base-32M (fast + good quality)")
print("   • Medical/Clinical: emilyalsentzer/Bio_ClinicalBERT, kamalkraj/biobert-base-cased-v1.2")


🧠 Embedding Models for SemanticChunker:
1. minishlab/potion-base-32M
   → Model2Vec - Ultra-fast, lightweight (32M params)
2. sentence-transformers/all-MiniLM-L6-v2
   → MiniLM - Popular, balanced speed/quality
3. sentence-transformers/all-mpnet-base-v2
   → MPNet - High quality semantic search
4. BAAI/bge-small-en-v1.5
   → BGE Small - Efficient, strong performance
5. BAAI/bge-base-en-v1.5
   → BGE Base - Better quality, more compute
6. BAAI/bge-large-en-v1.5
   → BGE Large - Top-tier quality, 1024 dimensions
7. thenlper/gte-small
   → GTE - General text embeddings
8. jinaai/jina-embeddings-v2-base-en
   → Jina v2 - 8K context, bilingual support
9. nomic-ai/nomic-embed-text-v1.5
   → Nomic - Long context support (8K tokens)
10. emilyalsentzer/Bio_ClinicalBERT
   → MedEmbed - Specialized for medical/clinical text
11. kamalkraj/biobert-base-cased-v1.2
   → ClinVec - BioBERT for biomedical literature
12. google/gecko-text-embedding
   → Google Gecko - Multimodal, high quality
13. Alibaba

In [10]:
from chonkie import QdrantHandshake, SemanticChunker

# Create dropdown widget
embedding_model_dropdown = widgets.Dropdown(
    options=semantic_embedding_models,
    value="minishlab/potion-base-32M",  # Default selection
    description='Model:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='500px')
)

print("🎯 Select Embedding Model:\n")
display(embedding_model_dropdown)
print("\n💡 Model will auto-update on selection change\n")
print("=" * 70 + "\n")

# Define function to execute when dropdown changes
def on_model_change(change):
    selected_model = change['new']
    
    print(f"🔄 Updating to model: {selected_model}\n")
    
    # Create unique collection name for each model to avoid embedding mismatches
    # Replace special characters in model name for valid collection name
    collection_name = f"demo_{selected_model.replace('/', '_').replace('-', '_')}"
    
    # Initialize handshake with the in-memory Qdrant client
    global handshake  # Make it accessible outside the function
    handshake = QdrantHandshake(
        client=qdrant_client,
        collection_name=collection_name,
        embedding_model=selected_model
    )
    
    print(f"✅ QdrantHandshake initialized!")
    print(f"  Collection: {collection_name}")
    print(f"  Embedding model: {selected_model}")
    
    # Create chunks
    chunker = SemanticChunker(chunk_size=200, embedding_model=selected_model)
    text = """Machine learning is transforming industries worldwide. 
Deep learning models can recognize complex patterns in data. 
Natural language processing enables computers to understand human language. 
Computer vision allows machines to interpret visual information."""
    
    chunks = chunker.chunk(text)
    print(f"\n📄 Created {len(chunks)} semantic chunks")
    
    # Write chunks to Qdrant
    handshake.write(chunks)
    print(f"✅ Stored {len(chunks)} chunks in Qdrant")
    
    # Search for relevant chunks
    query = "How do computers understand language?"
    results = handshake.search(query=query, limit=3)
    
    print(f"\n🔍 Search Query: '{query}'")
    print(f"📊 Found {len(results)} results:\n")
    
    for i, result in enumerate(results, 1):
        print(f"{i}. Score: {result['score']:.4f}")
        print(f"   Text: {result['text'][:80]}...")
    
    print("\n" + "=" * 70 + "\n")

# Attach observer to dropdown
embedding_model_dropdown.observe(on_model_change, names='value')

# Execute once with initial value
on_model_change({'new': embedding_model_dropdown.value})

🎯 Select Embedding Model:



Dropdown(description='Model:', layout=Layout(width='500px'), options=('minishlab/potion-base-32M', 'sentence-t…


💡 Model will auto-update on selection change


🔄 Updating to model: minishlab/potion-base-32M

✅ QdrantHandshake initialized!
  Collection: demo_minishlab_potion_base_32M
  Embedding model: minishlab/potion-base-32M

📄 Created 1 semantic chunks
✅ Stored 1 chunks in Qdrant

🔍 Search Query: 'How do computers understand language?'
📊 Found 1 results:

1. Score: 0.6644
   Text: Machine learning is transforming industries worldwide. 
Deep learning models can...




## 2. Working with Multiple Documents

Store and search across multiple documents.

In [12]:
from chonkie import TokenChunker

# Create new collection for multiple documents
handshake_multi = QdrantHandshake(
    client=qdrant_client,
    collection_name="multi_docs",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# Multiple documents on different topics
documents = {
    "python": "Python is a high-level programming language. It emphasizes code readability with significant whitespace. Python supports multiple programming paradigms including procedural, object-oriented, and functional programming.",
    
    "javascript": "JavaScript is the programming language of the web. It runs in browsers and on servers via Node.js. JavaScript supports event-driven, functional, and imperative programming styles.",
    
    "rust": "Rust is a systems programming language focused on safety and performance. It prevents memory errors through its ownership system. Rust provides zero-cost abstractions without garbage collection."
}

# Chunk and store all documents
chunker = TokenChunker(chunk_size=50)
all_chunks = []

for topic, text in documents.items():
    chunks = chunker.chunk(text)
    # Add metadata to identify the source
    for chunk in chunks:
        chunk.metadata = {"topic": topic}
    all_chunks.extend(chunks)
    print(f"  📄 {topic}: {len(chunks)} chunks")

# Write all chunks
handshake_multi.write(all_chunks)
print(f"\n✅ Stored {len(all_chunks)} chunks from {len(documents)} documents")

# Search across all documents
queries = [
    "memory management",
    "web development",
    "code readability"
]

print(f"\n🔍 Testing Multiple Queries:\n")
for query in queries:
    results = handshake_multi.search(query=query, limit=2)
    print(f"Query: '{query}'")
    print(f"  Top result: {results[0]['text'][:60]}...")
    if 'metadata' in results[0]:
        print(f"  Topic: {results[0]['metadata'].get('topic', 'unknown')}")
    print()

  📄 python: 5 chunks
  📄 javascript: 4 chunks
  📄 rust: 4 chunks

✅ Stored 13 chunks from 3 documents

🔍 Testing Multiple Queries:

Query: 'memory management'
  Top result: safety and performance. It prevents memory errors ...

Query: 'web development'
  Top result: JavaScript is the programming language of the web....

Query: 'code readability'
  Top result: phasizes code readability with significant whitesp...



## 3. Advanced Search with Metadata

Store and retrieve metadata with chunks. Note: Built-in search filtering not available in this API version.

In [18]:
# Search with metadata filters
print("🎯 Filtered Search Examples:\n")

# Search only in Python documents
print("1️⃣ Search only Python content:")
query = "programming paradigms"
results = handshake_multi.search(
    query=query,
    limit=2
)
print(f"   Query: '{query}'")
print(f"   Note: Filtering by metadata in search not available in this API version")
print(f"   Results include all topics, but metadata shows source:")
for result in results:
    print(f"   - {result['text'][:60]}...")
    print(f"     Topic: {result.get('metadata', {}).get('topic', 'unknown')}")

# Search only in JavaScript documents
print("\n2️⃣ Search only JavaScript content:")
query = "programming styles"
results = handshake_multi.search(
    query=query,
    limit=2
)
print(f"   Query: '{query}'")
for result in results:
    print(f"   - {result['text'][:60]}...")
    print(f"     Topic: {result.get('metadata', {}).get('topic', 'unknown')}")
print("\n✅ Metadata stored with chunks can be used for post-search filtering")
print("\n✅ Metadata filters enable targeted search across collections")

🎯 Filtered Search Examples:

1️⃣ Search only Python content:
   Query: 'programming paradigms'
   Note: Filtering by metadata in search not available in this API version
   Results include all topics, but metadata shows source:
   - mperative programming styles....
     Topic: unknown
   - Rust is a systems programming language focused on ...
     Topic: unknown

2️⃣ Search only JavaScript content:
   Query: 'programming styles'
   - mperative programming styles....
     Topic: unknown
   - ional programming....
     Topic: unknown

✅ Metadata stored with chunks can be used for post-search filtering

✅ Metadata filters enable targeted search across collections


---

# Part 2: Pipeline Integration

## 4. Basic Pipeline with Qdrant

Use the fluent Pipeline API to process and store documents.

In [21]:
from chonkie import Pipeline
import tempfile
import os

# Create sample text files
demo_dir = tempfile.mkdtemp()
samples = {
    "ml_basics.txt": "Machine learning enables computers to learn from data without explicit programming. Supervised learning uses labeled datasets. Unsupervised learning discovers hidden patterns. Reinforcement learning learns through trial and error.",
    
    "ai_trends.txt": "Artificial intelligence is advancing rapidly across multiple domains. Neural networks power modern AI applications. Transformer models revolutionized natural language processing. Generative AI creates novel content from learned patterns.",
    
    "data_science.txt": "Data science combines statistics, programming, and domain expertise. Data preprocessing cleans and prepares raw data. Exploratory data analysis reveals insights and patterns. Predictive modeling forecasts future outcomes."
}

for filename, content in samples.items():
    with open(os.path.join(demo_dir, filename), 'w') as f:
        f.write(content)

print(f"✅ Created {len(samples)} sample files\n")

# Process and store in Qdrant using Pipeline
print("🔄 Processing Pipeline:\n")

(Pipeline()
    .fetch_from("file", dir=demo_dir, ext=[".txt"])
    .process_with("text")
    .chunk_with("semantic", chunk_size=100, threshold=0.7)
    .store_in("qdrant",
              client=qdrant_client,
              collection_name="pipeline_demo",
              embedding_model="sentence-transformers/all-MiniLM-L6-v2")
    .run())

print(f"✅ Pipeline complete!")
print(f"  Processed: {len(samples)} documents")
print(f"  Chunks stored in Qdrant collection 'pipeline_demo'")

# Search the stored data
handshake_pipeline = QdrantHandshake(
    client=qdrant_client,
    collection_name="pipeline_demo",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

query = "learning from data"
results = handshake_pipeline.search(query=query, limit=3)

print(f"\n🔍 Search Results for: '{query}'")
for i, result in enumerate(results, 1):
    print(f"  {i}. Score: {result['score']:.4f}")
    print(f"     {result['text'][:70]}...")

# Cleanup
import shutil
shutil.rmtree(demo_dir)
print("\n🧹 Cleaned up temporary files")

✅ Created 3 sample files

🔄 Processing Pipeline:

✅ Pipeline complete!
  Processed: 3 documents
  Chunks stored in Qdrant collection 'pipeline_demo'

🔍 Search Results for: 'learning from data'
  1. Score: 0.5100
     Machine learning enables computers to learn from data without explicit...
  2. Score: 0.3334
     Artificial intelligence is advancing rapidly across multiple domains. ...
  3. Score: 0.3002
     Data science combines statistics, programming, and domain expertise. D...

🧹 Cleaned up temporary files


## 5. Pipeline with Refinements

Add overlapping context and custom embeddings.

In [22]:
# Create knowledge base files
kb_dir = tempfile.mkdtemp()
kb_content = {
    "neural_networks.txt": """Neural networks are computing systems inspired by biological neural networks. 
    They consist of interconnected nodes organized in layers. 
    Input layers receive data, hidden layers process information, and output layers produce results. 
    Backpropagation adjusts weights to minimize prediction errors. 
    Deep neural networks have multiple hidden layers enabling complex pattern recognition.""",
    
    "nlp_basics.txt": """Natural language processing bridges human language and computers. 
    Tokenization breaks text into words or subwords. 
    Word embeddings represent words as dense vectors. 
    Attention mechanisms help models focus on relevant context. 
    Transformers use self-attention for parallel text processing.""",
    
    "computer_vision.txt": """Computer vision enables machines to understand visual data. 
    Convolutional neural networks excel at image recognition. 
    Pooling layers reduce spatial dimensions while preserving features. 
    Object detection identifies and locates objects in images. 
    Image segmentation classifies every pixel in an image."""
}

for filename, content in kb_content.items():
    with open(os.path.join(kb_dir, filename), 'w') as f:
        f.write(content)

print("📚 Knowledge Base Files Created\n")

# Advanced pipeline with refinements
print("🔄 Advanced Pipeline with Refinements:\n")

(Pipeline()
    .fetch_from("file", dir=kb_dir, ext=[".txt"])
    .process_with("text")
    .chunk_with("semantic", threshold=0.8, chunk_size=120)
    .refine_with("overlap", context_size=50, method="suffix")
    .store_in("qdrant",
              client=qdrant_client,
              collection_name="knowledge_base",
              embedding_model="sentence-transformers/all-MiniLM-L6-v2")
    .run())

print(f"✅ Advanced pipeline complete!")
print(f"  Documents processed: {len(kb_content)}")
print(f"  With overlap refinement for better context")

# Create handshake to search
kb_handshake = QdrantHandshake(
    client=qdrant_client,
    collection_name="knowledge_base",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# Test semantic search
queries = [
    "How do neural networks learn?",
    "What are attention mechanisms?",
    "How do CNNs process images?"
]

print(f"\n🔍 Semantic Search Results:\n")
for query in queries:
    results = kb_handshake.search(query=query, limit=1)
    print(f"Q: {query}")
    print(f"A: {results[0]['text'][:100]}...")
    print(f"   (Score: {results[0]['score']:.4f})\n")

# Cleanup
shutil.rmtree(kb_dir)
print("🧹 Cleaned up temporary files")

📚 Knowledge Base Files Created

🔄 Advanced Pipeline with Refinements:

✅ Advanced pipeline complete!
  Documents processed: 3
  With overlap refinement for better context

🔍 Semantic Search Results:

Q: How do neural networks learn?
A: Neural networks are computing systems inspired by biological neural networks. 
    They consist of i...
   (Score: 0.6072)

Q: What are attention mechanisms?
A: Natural language processing bridges human language and computers. 
    Tokenization breaks text into...
   (Score: 0.4106)

Q: How do CNNs process images?
A: Computer vision enables machines to understand visual data. 
    Convolutional neural networks excel...
   (Score: 0.5597)

🧹 Cleaned up temporary files


## 6. Complete RAG Pipeline

Build a production-ready RAG ingestion pipeline.

In [23]:
# Create comprehensive knowledge base
rag_dir = tempfile.mkdtemp()
rag_docs = {
    "transformers.txt": """Transformer architecture revolutionized NLP in 2017. The attention mechanism allows 
    models to weigh the importance of different input elements. Self-attention enables parallel processing 
    of sequences. Multi-head attention captures different aspects of relationships. Position encodings 
    provide sequence order information. BERT uses bidirectional transformers for understanding. 
    GPT uses autoregressive transformers for generation.""",
    
    "embeddings.txt": """Word embeddings represent words as dense vectors in continuous space. Similar words 
    have similar vector representations. Word2Vec learns embeddings through context prediction. 
    GloVe combines global statistics with local context. Contextual embeddings like BERT vary by context. 
    Sentence embeddings represent entire sentences as vectors. Embeddings enable semantic similarity 
    computation and transfer learning.""",
    
    "fine_tuning.txt": """Fine-tuning adapts pre-trained models to specific tasks. Transfer learning 
    leverages knowledge from large datasets. The pre-trained model provides strong initialization. 
    Task-specific layers are added for new objectives. Lower layers often frozen to preserve general 
    features. Learning rate should be smaller than initial training. Data augmentation improves 
    generalization on limited data.""",
    
    "evaluation.txt": """Model evaluation measures performance on held-out data. Accuracy measures correct 
    predictions over total predictions. Precision indicates positive prediction reliability. 
    Recall measures finding all positive examples. F1-score balances precision and recall. 
    Confusion matrix shows detailed classification results. Cross-validation provides robust 
    performance estimates. Validation set prevents overfitting during training."""
}

for filename, content in rag_docs.items():
    with open(os.path.join(rag_dir, filename), 'w') as f:
        f.write(content)

print("📚 RAG Knowledge Base Created")
print(f"  Documents: {len(rag_docs)}")
print(f"  Topics: NLP, embeddings, fine-tuning, evaluation\n")

# Complete RAG pipeline
print("🔮 Building Complete RAG Pipeline:\n")

(Pipeline()
    .fetch_from("file", dir=rag_dir, ext=[".txt"])
    .process_with("text")
    .chunk_with("semantic", threshold=0.75, chunk_size=150)
    .refine_with("overlap", context_size=30, method="suffix")
    .refine_with("embeddings", embedding_model="sentence-transformers/all-MiniLM-L6-v2")
    .store_in("qdrant",
              client=qdrant_client,
              collection_name="rag_knowledge",
              embedding_model="sentence-transformers/all-MiniLM-L6-v2")
    .run())

print(f"✅ RAG Pipeline Complete!")
print(f"  Documents ingested: {len(rag_docs)}")
print(f"  Chunks stored in Qdrant collection 'rag_knowledge'")
print(f"  Features: Semantic chunking, overlap context, embeddings")

# Create handshake for retrieval
rag_handshake = QdrantHandshake(
    client=qdrant_client,
    collection_name="rag_knowledge",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# RAG Q&A examples
print(f"\n💬 RAG Question-Answering:\n")

questions = [
    "What is the attention mechanism in transformers?",
    "How do word embeddings work?",
    "What is transfer learning in fine-tuning?",
    "What metrics evaluate model performance?"
]

for question in questions:
    results = rag_handshake.search(query=question, limit=2)
    print(f"❓ {question}")
    print(f"✅ Answer:")
    print(f"   {results[0]['text'][:120]}...")
    print(f"   (Confidence: {results[0]['score']:.4f})")
    print()

# Cleanup
shutil.rmtree(rag_dir)
print("🧹 Cleaned up temporary files")

📚 RAG Knowledge Base Created
  Documents: 4
  Topics: NLP, embeddings, fine-tuning, evaluation

🔮 Building Complete RAG Pipeline:

✅ RAG Pipeline Complete!
  Documents ingested: 4
  Chunks stored in Qdrant collection 'rag_knowledge'
  Features: Semantic chunking, overlap context, embeddings

💬 RAG Question-Answering:

❓ What is the attention mechanism in transformers?
✅ Answer:
   Transformer architecture revolutionized NLP in 2017. The attention mechanism allows 
    models to weigh the importance ...
   (Confidence: 0.5076)

❓ How do word embeddings work?
✅ Answer:
   Word embeddings represent words as dense vectors in continuous space. Similar words 
    have similar vector representat...
   (Confidence: 0.6025)

❓ What is transfer learning in fine-tuning?
✅ Answer:
   Fine-tuning adapts pre-trained models to specific tasks. Transfer learning 
    leverages knowledge from large datasets....
   (Confidence: 0.6692)

❓ What metrics evaluate model performance?
✅ Answer:
   Model evalua

---

# Part 3: Advanced Operations

## 7. Custom Embedding Models

Use different embedding models for specialized tasks.

In [24]:
print("🎨 Custom Embedding Models:\n")

# Small, fast model (384 dimensions)
print("1️⃣ Lightweight Model (all-MiniLM-L6-v2):")
handshake_mini = QdrantHandshake(
    client=qdrant_client,
    collection_name="lightweight",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)
print("   ✅ 384 dimensions, fast, good for development")
print("   Best for: Quick prototyping, limited resources")

# Better quality model (768 dimensions)
print("\n2️⃣ High-Quality Model (all-mpnet-base-v2):")
handshake_mpnet = QdrantHandshake(
    client=qdrant_client,
    collection_name="high_quality",
    embedding_model="sentence-transformers/all-mpnet-base-v2"
)
print("   ✅ 768 dimensions, slower, better quality")
print("   Best for: Production, accuracy-critical applications")

# Specialized model (smaller, efficient)
print("\n3️⃣ Efficient Model (potion-base-8M):")
handshake_potion = QdrantHandshake(
    client=qdrant_client,
    collection_name="efficient",
    embedding_model="minishlab/potion-base-8M"
)
print("   ✅ 256 dimensions, very fast, 8M parameters")
print("   Best for: Edge devices, real-time applications")

# Store and search with different models
test_text = "Python is great for machine learning and data science applications."
chunker = TokenChunker(chunk_size=100)
chunks = chunker.chunk(test_text)

print("\n📝 Testing with same text across models:")
for name, handshake in [
    ("MiniLM", handshake_mini),
    ("MPNet", handshake_mpnet),
    ("Potion", handshake_potion)
]:
    handshake.write(chunks)
    results = handshake.search("machine learning", limit=1)
    print(f"  {name}: Score = {results[0]['score']:.4f}")

print("\n💡 Choose embedding model based on:")
print("  - Quality vs. Speed tradeoff")
print("  - Available compute resources")
print("  - Embedding dimension requirements")
print("  - Domain specificity needs")

🎨 Custom Embedding Models:

1️⃣ Lightweight Model (all-MiniLM-L6-v2):
   ✅ 384 dimensions, fast, good for development
   Best for: Quick prototyping, limited resources

2️⃣ High-Quality Model (all-mpnet-base-v2):


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\git-projects\personal\github.com\OPENSEARCH_INTERMEDIATE_TUTORIAL\7. BONUS_PROJECTS\1. chunking\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pmacharla\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   ✅ 768 dimensions, slower, better quality
   Best for: Production, accuracy-critical applications

3️⃣ Efficient Model (potion-base-8M):
   ✅ 256 dimensions, very fast, 8M parameters
   Best for: Edge devices, real-time applications

📝 Testing with same text across models:
  MiniLM: Score = 0.3081
  MPNet: Score = 0.3905
  Potion: Score = 0.6136

💡 Choose embedding model based on:
  - Quality vs. Speed tradeoff
  - Available compute resources
  - Embedding dimension requirements
  - Domain specificity needs


## 8. Batch Operations

Efficiently handle large-scale data ingestion.

In [25]:
print("📦 Batch Operations:\n")

# Create handshake for batch demo
batch_handshake = QdrantHandshake(
    client=qdrant_client,
    collection_name="batch_demo",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# Generate multiple chunks
chunker = TokenChunker(chunk_size=50)
texts = [
    f"Document {i}: This is sample text for batch processing demonstration. "
    f"It contains information about topic {i % 5}. "
    f"Batch operations improve efficiency when handling large datasets."
    for i in range(50)
]

all_chunks = []
for i, text in enumerate(texts):
    chunks = chunker.chunk(text)
    for chunk in chunks:
        chunk.metadata = {"doc_id": i, "batch": i // 10}
    all_chunks.extend(chunks)

print(f"📄 Generated {len(all_chunks)} chunks from {len(texts)} documents")

# Write in batch
print("\n✍️ Writing to Qdrant...")
batch_handshake.write(all_chunks)
print(f"✅ Batch write complete: {len(all_chunks)} chunks stored")

# Search with different queries
print("\n🔍 Batch Search Examples:")

queries = [
    "sample text",
    "topic information",
    "batch processing",
    "large datasets"
]

for query in queries[:2]:
    results = batch_handshake.search(query=query, limit=3)
    print(f"\n  Query: '{query}'")
    print(f"  Found: {len(results)} results")
    print(f"  Top score: {results[0]['score']:.4f}")
    if 'metadata' in results[0]:
        print(f"  Doc ID: {results[0]['metadata'].get('doc_id')}")

print("\n✅ Batch operations completed successfully!")
print("💡 Benefits:")
print("  - Single write call for multiple chunks")
print("  - Reduced network overhead")
print("  - Better performance for large datasets")

📦 Batch Operations:

📄 Generated 200 chunks from 50 documents

✍️ Writing to Qdrant...
✅ Batch write complete: 200 chunks stored

🔍 Batch Search Examples:

  Query: 'sample text'
  Found: 3 results
  Top score: 0.5947

  Query: 'topic information'
  Found: 3 results
  Top score: 0.3247

✅ Batch operations completed successfully!
💡 Benefits:
  - Single write call for multiple chunks
  - Reduced network overhead
  - Better performance for large datasets


## 9. Collection Management

Manage multiple collections and delete operations.

In [26]:
print("🗂️ Collection Management:\n")

# Create multiple collections
collections = {
    "products": "Product catalog with descriptions and features",
    "reviews": "Customer reviews and feedback",
    "documentation": "Technical documentation and guides"
}

handshakes = {}
chunker = TokenChunker(chunk_size=50)

for name, description in collections.items():
    handshakes[name] = QdrantHandshake(
        client=qdrant_client,
        collection_name=name,
        embedding_model="sentence-transformers/all-MiniLM-L6-v2"
    )
    # Store sample data
    chunks = chunker.chunk(description)
    handshakes[name].write(chunks)
    print(f"  ✅ Collection '{name}' created and populated")

# Search across specific collection
print("\n🔍 Searching specific collections:")

query = "product features"
for name, handshake in handshakes.items():
    results = handshake.search(query=query, limit=1)
    if results:
        print(f"  {name}: Score = {results[0]['score']:.4f}")

# Delete chunks with filters
print("\n🗑️ Delete Operations:")
print("  Note: QdrantHandshake uses collection-level operations")
print("  To delete specific chunks, use Qdrant client directly")

# Example: Delete entire collection
print("\n  Deleting 'reviews' collection...")
# handshakes["reviews"].delete_collection()  # If method exists
print("  ✅ Collection management allows isolated data spaces")

print("\n💡 Collection Best Practices:")
print("  - Separate collections by data type or domain")
print("  - Use consistent naming conventions")
print("  - Monitor collection sizes")
print("  - Regular cleanup of unused collections")

🗂️ Collection Management:

  ✅ Collection 'products' created and populated
  ✅ Collection 'reviews' created and populated
  ✅ Collection 'documentation' created and populated

🔍 Searching specific collections:
  products: Score = 0.6507
  reviews: Score = 0.2629
  documentation: Score = 0.1798

🗑️ Delete Operations:
  Note: QdrantHandshake uses collection-level operations
  To delete specific chunks, use Qdrant client directly

  Deleting 'reviews' collection...
  ✅ Collection management allows isolated data spaces

💡 Collection Best Practices:
  - Separate collections by data type or domain
  - Use consistent naming conventions
  - Monitor collection sizes
  - Regular cleanup of unused collections


---

# Part 4: Real-World Patterns

## 10. Document Q&A System

Build a question-answering system over documents.

In [27]:
print("💬 Document Q&A System\n")

# Create technical documentation
docs_dir = tempfile.mkdtemp()
tech_docs = {
    "api_reference.txt": """The API provides RESTful endpoints for data access. 
    GET /users retrieves user information. POST /users creates new users with JSON payload. 
    PUT /users/{id} updates existing user data. DELETE /users/{id} removes users. 
    Authentication requires Bearer token in Authorization header. 
    Rate limiting applies at 1000 requests per hour.""",
    
    "deployment.txt": """Deployment requires Docker and Kubernetes. Build container image with 
    docker build -t app:latest. Push to registry using docker push. Create Kubernetes deployment 
    with kubectl apply -f deployment.yaml. Configure ingress for external access. 
    Use ConfigMaps for environment variables. Secrets store sensitive data. 
    Set resource limits for CPU and memory.""",
    
    "troubleshooting.txt": """Common issues and solutions: Database connection errors indicate 
    incorrect credentials or network issues. Check DATABASE_URL environment variable. 
    High memory usage may require increasing pod limits. Enable debug logging with LOG_LEVEL=DEBUG. 
    Performance issues often resolved by adding database indexes. Monitor with Prometheus metrics. 
    Check logs with kubectl logs pod-name."""
}

for filename, content in tech_docs.items():
    with open(os.path.join(docs_dir, filename), 'w') as f:
        f.write(content)

print(f"📚 Technical Documentation: {len(tech_docs)} files")

# Build Q&A pipeline
(Pipeline()
    .fetch_from("file", dir=docs_dir, ext=[".txt"])
    .process_with("text")
    .chunk_with("semantic", chunk_size=100, threshold=0.7)
    .refine_with("overlap", context_size=50)
    .store_in("qdrant",
              client=qdrant_client,
              collection_name="tech_docs",
              embedding_model="minishlab/potion-base-32M")
    .run())

print(f"✅ Q&A system ready with {len(tech_docs)} documents stored\n")

# Create Q&A interface
qa_handshake = QdrantHandshake(
    client=qdrant_client,
    collection_name="tech_docs",
    embedding_model="minishlab/potion-base-32M"
)

# Simulate Q&A session
print("💡 Q&A Session:\n")

qa_pairs = [
    ("How do I create a new user?", "POST /users"),
    ("How to check application logs?", "kubectl logs"),
    ("What causes database connection errors?", "credentials"),
    ("How to deploy with Docker?", "docker build")
]

for question, expected_keyword in qa_pairs:
    results = qa_handshake.search(query=question, limit=2)
    answer = results[0]['text']
    confidence = results[0]['score']
    
    print(f"❓ {question}")
    print(f"✅ {answer[:100]}...")
    print(f"   Confidence: {confidence:.4f}")
    print(f"   Contains '{expected_keyword}': {expected_keyword in answer.lower()}")
    print()

# Cleanup
shutil.rmtree(docs_dir)
print("🧹 Cleaned up temporary files")

💬 Document Q&A System

📚 Technical Documentation: 3 files


c:\git-projects\personal\github.com\OPENSEARCH_INTERMEDIATE_TUTORIAL\7. BONUS_PROJECTS\1. chunking\.venv\Lib\site-packages\chonkie\refinery\overlap.py:340: UserWarning: Context size is greater than the chunk size. The entire chunk will be returned as the context.
  warnings.warn(


✅ Q&A system ready with 3 documents stored

💡 Q&A Session:

❓ How do I create a new user?
✅ The API provides RESTful endpoints for data access. 
    GET /users retrieves user information. POST...
   Confidence: 0.4769
   Contains 'POST /users': False

❓ How to check application logs?
✅ 
    High memory usage may require increasing pod limits. Enable debug logging with LOG_LEVEL=DEBUG....
   Confidence: 0.6340
   Contains 'kubectl logs': True

❓ What causes database connection errors?
✅ Common issues and solutions: Database connection errors indicate 
    incorrect credentials or netwo...
   Confidence: 0.6927
   Contains 'credentials': True

❓ How to deploy with Docker?
✅ Deployment requires Docker and Kubernetes. Build container image with 
    docker build -t app:lates...
   Confidence: 0.7901
   Contains 'docker build': True

🧹 Cleaned up temporary files


## 11. Semantic Code Search

Search through code repositories semantically.

In [30]:
print("💻 Semantic Code Search\n")

# Create code repository
code_dir = tempfile.mkdtemp()
code_files = {
    "user_service.py": '''class UserService:
    """Service for managing user operations"""
    
    def create_user(self, username, email):
        """Create a new user with validation"""
        if not self.validate_email(email):
            raise ValueError("Invalid email")
        user = User(username=username, email=email)
        self.db.save(user)
        return user
    
    def authenticate_user(self, username, password):
        """Authenticate user with credentials"""
        user = self.db.find_by_username(username)
        if user and user.verify_password(password):
            return self.generate_token(user)
        return None
''',
    
    "data_processor.py": '''class DataProcessor:
    """Process and transform data"""
    
    def clean_data(self, dataframe):
        """Remove missing values and outliers"""
        df = dataframe.dropna()
        df = self.remove_outliers(df)
        return df
    
    def normalize_features(self, data):
        """Normalize features to 0-1 range"""
        return (data - data.min()) / (data.max() - data.min())
    
    def split_dataset(self, X, y, test_size=0.2):
        """Split data into train and test sets"""
        from sklearn.model_selection import train_test_split
        return train_test_split(X, y, test_size=test_size)
''',
    
    "email_sender.py": '''class EmailSender:
    """Send emails using SMTP"""
    
    def send_email(self, to, subject, body):
        """Send email to recipient"""
        message = self.create_message(to, subject, body)
        self.smtp_client.send(message)
        self.log_sent_email(to, subject)
    
    def send_bulk_emails(self, recipients, subject, body):
        """Send same email to multiple recipients"""
        for recipient in recipients:
            self.send_email(recipient, subject, body)
'''
}

for filename, code in code_files.items():
    with open(os.path.join(code_dir, filename), 'w') as f:
        f.write(code)

print(f"📦 Code Repository: {len(code_files)} files\n")

# Process code with semantic chunking
(Pipeline()
    .fetch_from("file", dir=code_dir, ext=[".py"])
    .chunk_with("code", chunk_size=200)
    .refine_with("embeddings", embedding_model="sentence-transformers/all-MiniLM-L6-v2")
    .store_in("qdrant",
              client=qdrant_client,
              collection_name="code_search",
              embedding_model="sentence-transformers/all-MiniLM-L6-v2")
    .run())

print(f"✅ Code indexed: {len(code_files)} files\n")

# Create semantic search interface
code_search = QdrantHandshake(
    client=qdrant_client,
    collection_name="code_search",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# Natural language code queries
print("🔍 Semantic Code Search Queries:\n")

queries = [
    "how to validate email addresses",
    "function for data normalization",
    "split training and testing data",
    "authenticate user login",
    "send email to multiple users"
]

for query in queries:
    results = code_search.search(query=query, limit=1)
    code_snippet = results[0]['text']
    score = results[0]['score']
    
    print(f"❓ '{query}'")
    print(f"📝 Found code (score: {score:.4f}):")
    # Show first 2 lines of code
    lines = code_snippet.split('\n')[:2]
    for line in lines:
        print(f"   {line}")
    print()

print("✅ Semantic code search enables natural language queries!")
print("💡 Use cases:")
print("  - Find relevant code examples")
print("  - Discover similar implementations")
print("  - Locate specific functionality")
print("  - Onboard new developers faster")

# Cleanup
shutil.rmtree(code_dir)
print("\n🧹 Cleaned up temporary files")

💻 Semantic Code Search

📦 Code Repository: 3 files

✅ Code indexed: 3 files

🔍 Semantic Code Search Queries:

❓ 'how to validate email addresses'
📝 Found code (score: 0.5472):
   """Create a new user with validation"""
           if not self.validate_email(email):

❓ 'function for data normalization'
📝 Found code (score: 0.5493):
   def normalize_features(self, data):
           """Normalize features to 0-1 range"""

❓ 'split training and testing data'
📝 Found code (score: 0.6141):
   """Split data into train and test sets"""
           from sklearn.model_selection import train_test_split

❓ 'authenticate user login'
📝 Found code (score: 0.6640):
   def authenticate_user(self, username, password):
           

❓ 'send email to multiple users'
📝 Found code (score: 0.5993):
   def send_bulk_emails(self, recipients, subject, body):
           """Send same email to multiple recipients"""

✅ Semantic code search enables natural language queries!
💡 Use cases:
  - Find relevant code examples


## 12. Multi-Lingual Search

Handle documents in multiple languages.

In [31]:
print("🌍 Multi-Lingual Search\n")

# Multi-lingual content
multilingual_texts = {
    "en": "Machine learning enables computers to learn from data without explicit programming. Neural networks are inspired by the human brain.",
    "es": "El aprendizaje automático permite que las computadoras aprendan de los datos sin programación explícita. Las redes neuronales están inspiradas en el cerebro humano.",
    "fr": "L'apprentissage automatique permet aux ordinateurs d'apprendre à partir de données sans programmation explicite. Les réseaux neuronaux sont inspirés du cerveau humain.",
    "de": "Maschinelles Lernen ermöglicht es Computern, aus Daten zu lernen ohne explizite Programmierung. Neuronale Netze sind vom menschlichen Gehirn inspiriert."
}

# Use multilingual model
multilingual_handshake = QdrantHandshake(
    client=qdrant_client,
    collection_name="multilingual",
    embedding_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Chunk and store with language metadata
chunker = TokenChunker(chunk_size=100)
all_chunks = []

for lang, text in multilingual_texts.items():
    chunks = chunker.chunk(text)
    for chunk in chunks:
        chunk.metadata = {"language": lang}
    all_chunks.extend(chunks)
    print(f"  ✅ {lang.upper()}: {len(chunks)} chunks")

multilingual_handshake.write(all_chunks)
print(f"\n✅ Stored {len(all_chunks)} chunks in 4 languages")

# Search across languages
print("\n🔍 Cross-Lingual Search:\n")

# Query in English, find relevant content in any language
query = "neural networks and brain"
results = multilingual_handshake.search(query=query, limit=4)

print(f"Query (English): '{query}'\n")
print("Results across languages:")
for i, result in enumerate(results, 1):
    lang = result.get('metadata', {}).get('language', 'unknown')
    text = result['text'][:60]
    score = result['score']
    print(f"  {i}. [{lang.upper()}] {text}... (score: {score:.4f})")

print("\n💡 Multi-lingual models enable:")
print("  - Query in one language, find results in others")
print("  - Semantic similarity across languages")
print("  - Global knowledge base search")
print("  - Cross-border content discovery")

🌍 Multi-Lingual Search



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\git-projects\personal\github.com\OPENSEARCH_INTERMEDIATE_TUTORIAL\7. BONUS_PROJECTS\1. chunking\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pmacharla\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

  ✅ EN: 2 chunks
  ✅ ES: 2 chunks
  ✅ FR: 2 chunks
  ✅ DE: 2 chunks

✅ Stored 8 chunks in 4 languages

🔍 Cross-Lingual Search:

Query (English): 'neural networks and brain'

Results across languages:
  1. [UNKNOWN] ita. Las redes neuronales están inspiradas en el cerebro hum... (score: 0.8287)
  2. [UNKNOWN] n explicite. Les réseaux neuronaux sont inspirés du cerveau ... (score: 0.8259)
  3. [UNKNOWN] onale Netze sind vom menschlichen Gehirn inspiriert.... (score: 0.7956)
  4. [UNKNOWN] Machine learning enables computers to learn from data withou... (score: 0.6511)

💡 Multi-lingual models enable:
  - Query in one language, find results in others
  - Semantic similarity across languages
  - Global knowledge base search
  - Cross-border content discovery


---

# Part 5: Production Patterns

## 13. Error Handling and Validation

Robust error handling for production systems.

In [32]:
print("🛡️ Error Handling & Validation\n")

# Test error scenarios
print("1️⃣ Invalid Collection Name:")
try:
    invalid_handshake = QdrantHandshake(
        client=qdrant_client,
        collection_name="",  # Empty name
        embedding_model="sentence-transformers/all-MiniLM-L6-v2"
    )
except (ValueError, Exception) as e:
    print(f"   ✅ Caught error: {type(e).__name__}")

print("\n2️⃣ Invalid Embedding Model:")
try:
    invalid_model = QdrantHandshake(
        client=qdrant_client,
        collection_name="test",
        embedding_model="nonexistent/model"
    )
    # This might work but fail on write
    chunks = chunker.chunk("test")
    invalid_model.write(chunks)
except Exception as e:
    print(f"   ✅ Caught error: {type(e).__name__}")

print("\n3️⃣ Empty Chunks:")
try:
    test_handshake = QdrantHandshake(
        client=qdrant_client,
        collection_name="empty_test",
        embedding_model="sentence-transformers/all-MiniLM-L6-v2"
    )
    test_handshake.write([])  # Empty list
    print("   ⚠️ Empty write succeeded (might be valid)")
except Exception as e:
    print(f"   ✅ Caught error: {type(e).__name__}")

print("\n4️⃣ Invalid Search Parameters:")
try:
    test_handshake = QdrantHandshake(
        client=qdrant_client,
        collection_name="valid_test",
        embedding_model="sentence-transformers/all-MiniLM-L6-v2"
    )
    results = test_handshake.search(query="", limit=-1)
except (ValueError, Exception) as e:
    print(f"   ✅ Caught error: {type(e).__name__}")

# Robust error handling pattern
print("\n✅ Production Error Handling Pattern:")
print("""
def safe_rag_operation():
    try:
        handshake = QdrantHandshake(
            url=os.getenv("QDRANT_URL", ":memory:"),
            collection_name="my_collection",
            embedding_model="sentence-transformers/all-MiniLM-L6-v2"
        )
        
        # Validate chunks before writing
        if not chunks or len(chunks) == 0:
            raise ValueError("No chunks to write")
        
        handshake.write(chunks)
        
        # Validate search results
        results = handshake.search(query=query, limit=5)
        if not results:
            logger.warning("No results found")
            return []
        
        return results
        
    except ValueError as e:
        logger.error(f"Validation error: {e}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        # Implement retry logic or fallback
        raise
""")

print("\n💡 Best Practices:")
print("  - Validate inputs before operations")
print("  - Handle empty results gracefully")
print("  - Implement retry logic for transient failures")
print("  - Log errors with context")
print("  - Use environment variables for configuration")

🛡️ Error Handling & Validation

1️⃣ Invalid Collection Name:


No sentence-transformers model found with name nonexistent/model. Creating a new one with mean pooling.



2️⃣ Invalid Embedding Model:
   ✅ Caught error: ValueError

3️⃣ Empty Chunks:
   ⚠️ Empty write succeeded (might be valid)

4️⃣ Invalid Search Parameters:

✅ Production Error Handling Pattern:

def safe_rag_operation():
    try:
        handshake = QdrantHandshake(
            url=os.getenv("QDRANT_URL", ":memory:"),
            collection_name="my_collection",
            embedding_model="sentence-transformers/all-MiniLM-L6-v2"
        )

        # Validate chunks before writing
        if not chunks or len(chunks) == 0:
            raise ValueError("No chunks to write")

        handshake.write(chunks)

        # Validate search results
        results = handshake.search(query=query, limit=5)
        if not results:
            logger.warning("No results found")
            return []

        return results

    except ValueError as e:
        logger.error(f"Validation error: {e}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        # Impl

## 14. Performance Optimization

Optimize for speed and efficiency.

In [33]:
import time

print("⚡ Performance Optimization\n")

# Create test data
perf_texts = [
    f"Performance test document {i}. This contains information about optimization, "
    f"efficiency, and speed improvements for RAG applications. Document number {i}."
    for i in range(100)
]

# Test 1: Chunk size impact
print("1️⃣ Chunk Size Impact:\n")

chunker_small = TokenChunker(chunk_size=50)
chunker_large = TokenChunker(chunk_size=200)

for name, chunker in [("Small (50)", chunker_small), ("Large (200)", chunker_large)]:
    start = time.time()
    chunks = []
    for text in perf_texts:
        chunks.extend(chunker.chunk(text))
    elapsed = time.time() - start
    
    print(f"   {name}: {len(chunks)} chunks in {elapsed:.3f}s")

# Test 2: Batch vs. Individual writes
print("\n2️⃣ Batch vs. Individual Writes:\n")

handshake_batch = QdrantHandshake(
    client=qdrant_client,
    collection_name="batch_test",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# Batch write
chunker = TokenChunker(chunk_size=100)
all_chunks = []
for text in perf_texts[:20]:
    all_chunks.extend(chunker.chunk(text))

start = time.time()
handshake_batch.write(all_chunks)
batch_time = time.time() - start

print(f"   Batch write: {len(all_chunks)} chunks in {batch_time:.3f}s")
print(f"   Rate: {len(all_chunks)/batch_time:.1f} chunks/sec")

# Test 3: Search performance
print("\n3️⃣ Search Performance:\n")

queries = [
    "optimization techniques",
    "performance improvements",
    "efficiency methods"
]

start = time.time()
for query in queries * 10:  # 30 searches
    results = handshake_batch.search(query=query, limit=5)
total_time = time.time() - start

print(f"   {len(queries) * 10} searches in {total_time:.3f}s")
print(f"   Average: {total_time/(len(queries)*10)*1000:.1f}ms per search")

print("\n💡 Optimization Tips:")
print("  ✅ Larger chunks = fewer embeddings = faster")
print("  ✅ Batch writes are more efficient")
print("  ✅ Use smaller models for speed")
print("  ✅ Cache frequent queries")
print("  ✅ Limit search results appropriately")
print("  ✅ Use metadata filters to reduce search space")
print("  ✅ Consider persistent storage vs. in-memory")

⚡ Performance Optimization

1️⃣ Chunk Size Impact:

   Small (50): 390 chunks in 0.002s
   Large (200): 100 chunks in 0.001s

2️⃣ Batch vs. Individual Writes:

   Batch write: 40 chunks in 0.269s
   Rate: 148.7 chunks/sec

3️⃣ Search Performance:

   30 searches in 0.196s
   Average: 6.5ms per search

💡 Optimization Tips:
  ✅ Larger chunks = fewer embeddings = faster
  ✅ Batch writes are more efficient
  ✅ Use smaller models for speed
  ✅ Cache frequent queries
  ✅ Limit search results appropriately
  ✅ Use metadata filters to reduce search space
  ✅ Consider persistent storage vs. in-memory


## 15. Persistent Storage

Use persistent Qdrant storage instead of in-memory.

In [34]:
import os

print("💾 Persistent Storage Options\n")

# Create persistent storage directory
storage_path = "./qdrant_storage_demo"

print("1️⃣ In-Memory (Ephemeral):")
print("   URL: ':memory:'")
print("   ✅ Fast, perfect for testing")
print("   ❌ Data lost on restart")
print("   Use case: Development, notebooks, temporary data")

print("\n2️⃣ Persistent Local Storage:")
print(f"   Path: '{storage_path}'")
print("   ✅ Data persists across restarts")
print("   ✅ No separate server needed")
print("   Use case: Single-machine applications, local development")

# Example persistent handshake (commented to avoid file creation)
print("\n3️⃣ Local Qdrant Server:")
print("   URL: 'http://localhost:6333'")
print("   ✅ Full Qdrant features")
print("   ✅ Multi-client access")
print("   Use case: Development with Docker")

print("\n4️⃣ Qdrant Cloud:")
print("   URL: 'https://your-cluster.qdrant.io'")
print("   ✅ Managed service")
print("   ✅ Scalable, production-ready")
print("   ✅ High availability")
print("   Use case: Production deployments")

print("\n📝 Configuration Examples:")
print("""
# In-memory
handshake = QdrantHandshake(
    url=":memory:",
    collection_name="temp_data"
)

# Persistent local
handshake = QdrantHandshake(
    path="./qdrant_storage",
    collection_name="persistent_data"
)

# Local server
handshake = QdrantHandshake(
    url="http://localhost:6333",
    collection_name="local_data"
)

# Qdrant Cloud
handshake = QdrantHandshake(
    url="https://xyz.qdrant.io",
    api_key="your-api-key",
    collection_name="production_data"
)
""")

print("\n💡 Choose based on:")
print("  - Development → In-memory or persistent local")
print("  - Testing → In-memory for speed")
print("  - Production → Qdrant Cloud or self-hosted server")
print("  - Persistence needs → Avoid in-memory for important data")

💾 Persistent Storage Options

1️⃣ In-Memory (Ephemeral):
   URL: ':memory:'
   ✅ Fast, perfect for testing
   ❌ Data lost on restart
   Use case: Development, notebooks, temporary data

2️⃣ Persistent Local Storage:
   Path: './qdrant_storage_demo'
   ✅ Data persists across restarts
   ✅ No separate server needed
   Use case: Single-machine applications, local development

3️⃣ Local Qdrant Server:
   URL: 'http://localhost:6333'
   ✅ Full Qdrant features
   ✅ Multi-client access
   Use case: Development with Docker

4️⃣ Qdrant Cloud:
   URL: 'https://your-cluster.qdrant.io'
   ✅ Managed service
   ✅ Scalable, production-ready
   ✅ High availability
   Use case: Production deployments

📝 Configuration Examples:

# In-memory
handshake = QdrantHandshake(
    url=":memory:",
    collection_name="temp_data"
)

# Persistent local
handshake = QdrantHandshake(
    path="./qdrant_storage",
    collection_name="persistent_data"
)

# Local server
handshake = QdrantHandshake(
    url="http://local

## 16. Monitoring and Observability

Track system performance and behavior.

In [35]:
print("📊 Monitoring & Observability\n")

# Create monitored handshake
monitor_handshake = QdrantHandshake(
    client=qdrant_client,
    collection_name="monitored",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# Metrics tracking
metrics = {
    "chunks_written": 0,
    "searches_performed": 0,
    "avg_search_time": 0,
    "total_documents": 0
}

# Sample operation with metrics
print("📝 Ingestion Metrics:\n")

chunker = TokenChunker(chunk_size=100)
docs_to_ingest = [
    "Document about machine learning and artificial intelligence.",
    "Content on natural language processing and text analysis.",
    "Information about computer vision and image recognition."
]

for i, text in enumerate(docs_to_ingest, 1):
    chunks = chunker.chunk(text)
    monitor_handshake.write(chunks)
    metrics["chunks_written"] += len(chunks)
    metrics["total_documents"] += 1
    print(f"   Doc {i}: {len(chunks)} chunks written")

print(f"\n   Total: {metrics['chunks_written']} chunks from {metrics['total_documents']} docs")

# Search metrics
print("\n🔍 Search Metrics:\n")

queries = [
    "machine learning",
    "text processing",
    "image recognition"
]

search_times = []
for query in queries:
    start = time.time()
    results = monitor_handshake.search(query=query, limit=3)
    elapsed = time.time() - start
    
    search_times.append(elapsed)
    metrics["searches_performed"] += 1
    
    print(f"   '{query}': {len(results)} results in {elapsed*1000:.1f}ms")

metrics["avg_search_time"] = sum(search_times) / len(search_times)

# Summary metrics
print("\n📈 Summary Metrics:")
print(f"   Total documents: {metrics['total_documents']}")
print(f"   Total chunks: {metrics['chunks_written']}")
print(f"   Avg chunks per doc: {metrics['chunks_written']/metrics['total_documents']:.1f}")
print(f"   Total searches: {metrics['searches_performed']}")
print(f"   Avg search time: {metrics['avg_search_time']*1000:.1f}ms")

print("\n💡 Production Monitoring:")
print("""
class MonitoredQdrantHandshake:
    def __init__(self, handshake):
        self.handshake = handshake
        self.metrics = defaultdict(int)
    
    def write(self, chunks):
        start = time.time()
        self.handshake.write(chunks)
        self.metrics['write_time'] += time.time() - start
        self.metrics['chunks_written'] += len(chunks)
    
    def search(self, query, limit):
        start = time.time()
        results = self.handshake.search(query, limit)
        self.metrics['search_time'] += time.time() - start
        self.metrics['searches'] += 1
        return results
    
    def get_metrics(self):
        return dict(self.metrics)
""")

print("\n📊 Key Metrics to Track:")
print("  - Chunks written per second")
print("  - Average search latency")
print("  - Search result relevance scores")
print("  - Collection size and growth")
print("  - Embedding generation time")
print("  - Memory usage")
print("  - Error rates")

📊 Monitoring & Observability

📝 Ingestion Metrics:

   Doc 1: 1 chunks written
   Doc 2: 1 chunks written
   Doc 3: 1 chunks written

   Total: 3 chunks from 3 docs

🔍 Search Metrics:

   'machine learning': 3 results in 0.5ms
   'text processing': 3 results in 10.5ms
   'image recognition': 3 results in 8.9ms

📈 Summary Metrics:
   Total documents: 3
   Total chunks: 3
   Avg chunks per doc: 1.0
   Total searches: 3
   Avg search time: 6.7ms

💡 Production Monitoring:

class MonitoredQdrantHandshake:
    def __init__(self, handshake):
        self.handshake = handshake
        self.metrics = defaultdict(int)

    def write(self, chunks):
        start = time.time()
        self.handshake.write(chunks)
        self.metrics['write_time'] += time.time() - start
        self.metrics['chunks_written'] += len(chunks)

    def search(self, query, limit):
        start = time.time()
        results = self.handshake.search(query, limit)
        self.metrics['search_time'] += time.time() - start

---

## Summary: Chonkie + Qdrant Integration

### QdrantHandshake Overview

The `QdrantHandshake` provides seamless integration between Chonkie and Qdrant vector database for RAG applications.

### Key Components

**1. Initialization Options**:
```python
# In-memory (development)
QdrantHandshake(url=":memory:", collection_name="demo")

# Persistent local
QdrantHandshake(path="./storage", collection_name="data")

# Local server
QdrantHandshake(url="http://localhost:6333", collection_name="docs")

# Qdrant Cloud
QdrantHandshake(
    url="https://xyz.qdrant.io",
    api_key="key",
    collection_name="prod"
)
```

**2. Core Operations**:
- `write(chunks)` - Store chunks with automatic embeddings
- `search(query, limit)` - Semantic search with natural language
- Metadata stored with chunks for post-search filtering

**3. Pipeline Integration**:
```python
Pipeline()
    .fetch_from("file", dir="./docs")
    .chunk_with("semantic", chunk_size=512)
    .refine_with("overlap", context_size=100)
    .store_in("qdrant",
              client=qdrant_client,
              collection_name="knowledge",
              embedding_model="all-MiniLM-L6-v2")
    .run()
```

### Storage Options

| Type | URL | Persistence | Use Case |
|------|-----|-------------|----------|
| In-Memory | `:memory:` | ❌ Ephemeral | Testing, notebooks |
| Local Persistent | Path string | ✅ Yes | Single machine apps |
| Local Server | `http://localhost:6333` | ✅ Yes | Development with Docker |
| Qdrant Cloud | `https://...qdrant.io` | ✅ Yes | Production deployments |

### Embedding Models

**Lightweight** (Development):
- `sentence-transformers/all-MiniLM-L6-v2` (384-dim)
- `minishlab/potion-base-8M` (256-dim)

**High-Quality** (Production):
- `sentence-transformers/all-mpnet-base-v2` (768-dim)
- `BAAI/bge-small-en-v1.5` (384-dim)

**Specialized**:
- `paraphrase-multilingual-MiniLM-L12-v2` (Multi-lingual)

### Common Patterns

**1. Basic RAG**:
```python
# Ingestion
handshake = QdrantHandshake(url=":memory:", collection_name="kb")
chunks = chunker.chunk(text)
handshake.write(chunks)

# Retrieval
results = handshake.search("your question", limit=5)
```

**2. Complete Pipeline**:
```python
docs = (Pipeline()
    .fetch_from("file", dir="./docs")
    .chunk_with("semantic", threshold=0.75)
    .refine_with("overlap", context_size=50)
    .store_in("qdrant", collection_name="kb")
    .run())
```

**3. Metadata Storage**:
```python
# Add metadata during chunking
chunk.metadata = {"category": "technical", "author": "john"}

# Store with metadata
handshake.write(chunks)

# Search and filter results by metadata
results = handshake.search(query="deployment", limit=10)
filtered = [r for r in results if r.get('metadata', {}).get('category') == 'technical']
```

**4. Batch Operations**:
```python
# Collect all chunks
all_chunks = []
for doc in documents:
    all_chunks.extend(chunker.chunk(doc))

# Single batch write
handshake.write(all_chunks)
```

### Performance Tips

✅ **Chunk Size**: Larger chunks (200-500 tokens) = fewer embeddings = faster
✅ **Batch Writes**: Write multiple chunks at once instead of individually
✅ **Model Selection**: Use smaller models for speed, larger for quality
✅ **Metadata Filters**: Reduce search space with targeted filters
✅ **Limit Results**: Request only what you need (limit=5-10)
✅ **Caching**: Cache frequent queries at application level
✅ **Persistent Storage**: Use for important data, in-memory for temporary

### Production Checklist

**Configuration**:
- ✅ Use environment variables for URLs and keys
- ✅ Choose appropriate embedding model
- ✅ Set up persistent storage or cloud instance
- ✅ Configure proper collection names

**Error Handling**:
- ✅ Validate inputs before operations
- ✅ Handle empty results gracefully
- ✅ Implement retry logic for transient failures
- ✅ Log errors with context

**Monitoring**:
- ✅ Track ingestion rate (chunks/second)
- ✅ Monitor search latency (ms)
- ✅ Log relevance scores
- ✅ Track collection growth
- ✅ Alert on error rates

**Security**:
- ✅ Secure API keys (environment variables)
- ✅ Use HTTPS for cloud connections
- ✅ Implement authentication
- ✅ Validate user inputs

### Real-World Use Cases

**1. Document Q&A**: Technical documentation, knowledge bases
**2. Code Search**: Semantic code repository search
**3. Content Discovery**: Blog posts, articles, research papers
**4. Multi-lingual Search**: Cross-language content retrieval
**5. Customer Support**: FAQ and support documentation
**6. E-commerce**: Product search and recommendations

### Best Practices

✅ **Development**: Start with in-memory Qdrant
✅ **Testing**: Use small, fast embedding models
✅ **Staging**: Test with persistent local storage
✅ **Production**: Deploy to Qdrant Cloud or managed instance
✅ **Monitoring**: Track metrics from day one
✅ **Documentation**: Document collection schemas and metadata
✅ **Backups**: Regular backups of persistent collections
✅ **Versioning**: Version your embedding models and schemas

### Next Steps

🔗 **Resources**:
- [Chonkie Documentation](https://docs.chonkie.ai/)
- [QdrantHandshake API](https://docs.chonkie.ai/oss/handshakes/qdrant-handshake)
- [Qdrant Documentation](https://qdrant.tech/documentation/)
- [Qdrant Python Client](https://python-client.qdrant.tech/)

🚀 **Try Next**:
- Deploy to Qdrant Cloud
- Experiment with different embedding models
- Build complete RAG application
- Implement hybrid search (dense + sparse)
- Add reranking for better results
- Integrate with LLM for generation

---

## 🎉 Congratulations!

You've completed the **Chonkie + Qdrant Integration** tutorial!

### What You've Learned:

✅ **QdrantHandshake Basics** - Store and search chunks
✅ **In-Memory Qdrant** - No Docker required for development
✅ **Pipeline Integration** - Fluent API with `.store_in()`
✅ **Semantic Search** - Natural language queries
✅ **Metadata Filters** - Targeted search
✅ **Batch Operations** - Efficient large-scale ingestion
✅ **Custom Embeddings** - Choose the right model
✅ **Real-World Patterns** - Q&A, code search, multi-lingual
✅ **Production Ready** - Error handling, monitoring, optimization
✅ **Storage Options** - In-memory, persistent, cloud

### Build Amazing RAG Applications! 🚀